In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

import pandas as pd
import torch

import sys
sys.path.insert(1, "..")

from utils_llama import *

In [ ]:
from unsloth import FastLanguageModel
from datasets import load_dataset, Dataset, DatasetDict

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [ ]:
max_seq_length = 800 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.2-bnb-4bit",
    "unsloth/llama-2-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",
    "unsloth/gemma-7b-it-bnb-4bit", # Instruct version of Gemma 7b
    "unsloth/gemma-2b-bnb-4bit",
    "unsloth/gemma-2b-it-bnb-4bit", # Instruct version of Gemma 2b
    "unsloth/llama-3-8b-bnb-4bit", # [NEW] 15 Trillion token Llama-3
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

In [ ]:
df = pd.read_json('unsloth_input_last-phdos_all-bonds_only_robocrys.json')

In [ ]:
dataset_loaded = Dataset.from_pandas(df, preserve_index=False)

In [ ]:
train_test = dataset_loaded.train_test_split(test_size=.20, shuffle=True, seed=42)
train_valid = train_test["train"].train_test_split(test_size=0.20, shuffle=True, seed=42)

dataset = DatasetDict({"train": train_valid["train"],
                       "valid": train_valid["test"],
                      "test": train_test["test"]})

In [ ]:
dataset

In [ ]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

In [ ]:
dataset_train = dataset['train'].map(formatting_prompts_func, batched = True)
dataset_valid = dataset['valid'].map(formatting_prompts_func, batched = True)

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

In [ ]:
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset_train,
    eval_dataset = dataset_valid,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        evaluation_strategy="epoch",
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 10,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        save_strategy="epoch",
        load_best_model_at_end=True,
    ),
)

In [ ]:
trainer_stats = trainer.train()

In [ ]:
# Reload models from checkpoints and find one with lowest MAE of validation set
import os
checkpoint_dir = os.listdir("outputs")
maes_valid = {}
for model_dir in checkpoint_dir:
    model, tokenizer = FastLanguageModel.from_pretrained(os.path.join("outputs", model_dir))

    y_true, y_pred = get_prediction_all_info(dataset=dataset["valid"], model=model, tokenizer=tokenizer)
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    maes_valid[model_dir] = compute_mae(y_true, y_pred)

print(maes_valid)
print({k:v for k, v in maes_valid.items() if v==min(list(maes_valid.values()))})

In [ ]:
model_dir = min(maes_valid, key=maes_valid.get)
print(model_dir)

model, tokenizer = FastLanguageModel.from_pretrained(os.path.join("outputs", model_dir))
sts = ["test", "valid", "train"]
error_dict = {st: {} for st in sts}
for st in sts:
    y_true, y_pred = get_prediction_all_info(dataset=dataset[st], model=model, tokenizer=tokenizer)
    error_dict[st]["y_true"] = y_true
    error_dict[st]["y_pred"] = y_pred

    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    error_dict[st]["mae"] = compute_mae(y_true, y_pred)
    error_dict[st]["rmse"] = compute_rmse(y_true, y_pred)

import json
with open("result_dict_best_model_lora_model_only_robocrys_save_epochs.json", "w") as f:
    json.dump(error_dict, f)

In [0]:
model.save_pretrained("lora_model_only_robocrys_save_epochs") # Local saving
tokenizer.save_pretrained("lora_model_only_robocrys_save_epochs")

In [0]:
# Save in gguf for GUI -> further see "Chatbot" folder
model.save_pretrained_gguf("model_only_robocrys_save_epochs_gguf", tokenizer,)